In [ ]:
import os, sys, zarr
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from os.path import abspath, dirname
from easi_fish import n5_metadata_utils as n5mu
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## input
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
out_dir = os.path.join(input_dir, "testfull-aug5")  

# fixed image (directory - n5 format)
fix_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# get appropriate image data within fix_dir
subpath='\\c3\\s2'

# registered image (directory - n5 format)
reg_dir = os.path.join(input_dir, "r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped")  

# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 

# output directory
## end input

## output
out_mask = os.path.join(out_dir, 'mask.tif')
out_badroi = os.path.join(out_dir, 'bad_roi_list.npy') 

In [ ]:
%%time
#voxel size in µm (x, y, z) (post-expansion)
vox= n5mu.read_voxel_spacing(fix_dir, subpath)
#image size in pixel (x, y, z)
grid=n5mu.read_voxel_grid(fix_dir, subpath)
#image size in physical space (x, y, z) (post-expansion)
size=grid*vox
print('voxel size is:',vox)
print('image size in pixel unit is:',grid)
print('image size in um unit is:',size)

# get appropriate image data
print("loading images...")
fix = zarr.open(store=zarr.N5Store(fix_dir), mode='r')     
img1 = fix[subpath][:, :, :]

reg = zarr.open(store=zarr.N5Store(reg_dir), mode='r')     
img2 = reg[subpath][:, :, :]
print("all images loaded")

if not os.path.isdir(out_dir):
    print(out_dir)
    os.mkdir(out_dir)

mask=np.full((grid[2], grid[1], grid[0]),1)
mask[img2==0]=0
imsave(out_mask, mask)
print("mask generated")
print("mask dimension is:", mask.shape)

lb=imread(lb_dir)
roi = np.max(lb)
print(roi)

# # Get list of ROIs that are fully or partially outside the mask 
bad_roi=np.unique(lb[mask==0])
np.save(out_badroi, bad_roi)
print("# of ROIs rejected:", len(bad_roi))